In [1]:
import pandas as pd
import great_expectations as gx
from great_expectations.checkpoint.actions import UpdateDataDocsAction
import os
from datetime import datetime

In [2]:
# --- 1. CONFIGURAÇÃO E LEITURA ---
DATA_PATH = '../data/olist_processed.parquet'

print("Carregando dados processados com Pandas...")
df = pd.read_parquet(DATA_PATH)
print(f"Dados carregados: {df.shape[0]} linhas.")


Carregando dados processados com Pandas...
Dados carregados: 113314 linhas.


In [ ]:
# --- 1. Configuração do Great Expectations ---
print("1. Iniciando Contexto e Configurações...")
context = gx.get_context()

datasource_name = "pandas_datasource"
asset_name = "olist_processed"
batch_def_name = "batch_definition_olist"
suite_name = "minha_suite_de_testes"

try:
    # --- 2. Preparação da Estrutura ---
    print("2. Configurando Datasource...")
    data_source = context.data_sources.add_or_update_pandas(datasource_name)
    data_asset = data_source.add_dataframe_asset(name=asset_name)
    batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_def_name)

    # --- 3. Preparação da Suite ---
    print("3. Configurando Suite...")
    try:
        suite = context.suites.get(suite_name)
    except:
        suite = context.suites.add(gx.ExpectationSuite(name=suite_name))

    print("4. Gerando o Lote manualmente...")
    batch_parameters = {"dataframe": df}
    batch = batch_definition.get_batch(batch_parameters=batch_parameters)
    
    print("   -> Lote gerado com sucesso!")

    print("5. Criando Validador com Lote Pronto...")
    # Usar o parâmetro 'batch_list' para entregar o dado já pronto
    validator = context.get_validator(
        batch_list=[batch],
        expectation_suite=suite
    )

    print("\n✅ SUCESSO! O Validador está pronto.")

except Exception as e:
    print("\n❌ Ainda deu erro? Aqui está o detalhe:")
    print(e)

1. Iniciando Contexto e Configurações...
2. Configurando Datasource...
3. Configurando Suite...
4. Gerando o Lote manualmente...
   -> Lote gerado com sucesso!
5. Criando Validador com Lote Pronto...

✅ SUCESSO ABSOLUTO! O Validador está pronto.


In [ ]:
print("\n--- Iniciando Definição de Regras (Expectations) ---")
# --- 3. DEFININDO AS REGRAS DE QUALIDADE ---

# REGRA 1: Garantir que o ID do pedido é não nulo
validator.expect_column_values_to_not_be_null("order_id")

# REGRA 2: Validar status (se a coluna existir)
valid_statuses = ['delivered', 'shipped', 'canceled', 'invoiced', 'processing', 'unavailable', 'approved', 'created']
validator.expect_column_values_to_be_in_set("order_status", valid_statuses)

# REGRA 3: Preço e Frete não podem ser negativos
validator.expect_column_values_to_be_between("price", min_value=0)
validator.expect_column_values_to_be_between("freight_value", min_value=0)

# REGRA 4: Data não pode ser futura
if "order_purchase_timestamp" in df.columns:
    df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"], errors='coerce')
min_date = pd.Timestamp("2016-01-01")
max_date = pd.Timestamp(datetime.now().strftime("%Y-%m-%d"))

print(f"   Min: {min_date} | Max: {max_date}")

validator.expect_column_values_to_be_between(
    "order_purchase_timestamp", 
    min_value=min_date, 
    max_value=max_date,
)

# --- 4. RODANDO A VALIDAÇÃO ---
print("\nExecutando validações...")
validation_result = validator.validate()


C:\Users\luizm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\great_expectations\expectations\expectation.py:1633: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



--- Iniciando Definição de Regras (Expectations) ---


Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 240.00it/s]
C:\Users\luizm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\great_expectations\expectations\expectation.py:1633: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 106.79it/s]
C:\Users\luizm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\great_expectations\expectations\expectation.py:1633: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 153.26it/s]
C:\Users\luizm\AppData\L

   Min: 2016-01-01 00:00:00 | Max: 2025-12-31 00:00:00


Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 125.76it/s]



Executando validações...


Calculating Metrics: 100%|██████████| 29/29 [00:00<00:00, 141.48it/s]


In [ ]:
# 1. Salvando as Regras (Correção do erro de duplicidade)
print("1. Atualizando a Suite de Testes...")
suite_em_memoria = validator.get_expectation_suite()
context.suites.add_or_update(suite_em_memoria)

# 2. Recriando/Recuperando a infraestrutura do Checkpoint
print("2. Preparando o Checkpoint...")

# Primeiro, garantimos a Definição de Validação
validation_def_name = "validacao_olist_def"
try:
    val_def = context.validation_definitions.get(validation_def_name)
except:
    # Se não existir, cria de novo usando os objetos que temos
    val_def = context.validation_definitions.add(
        gx.ValidationDefinition(
            name=validation_def_name,
            data=batch_definition, 
            suite=suite_em_memoria
        )
    )

# Agora garantimos o Checkpoint
checkpoint_name = "checkpoint_olist_com_docs"
try:
    # Tenta pegar do contexto
    checkpoint = context.checkpoints.get(checkpoint_name)
except:
    # Se não existir, cria
    checkpoint = context.checkpoints.add(
        gx.Checkpoint(
            name=checkpoint_name,
            validation_definitions=[val_def],
            result_format={"result_format": "COMPLETE"},
            actions=[UpdateDataDocsAction(name="update_docs")]
        )
    )

# 3. Execução Final
print("3. Rodando Validação...")
results = checkpoint.run(
    batch_parameters={"dataframe": df} # Injeta o dataframe
)

print("4. Abrindo Relatório...")
try:
    context.open_data_docs()
    print("✅ Sucesso! Verifique a aba do navegador.")
except Exception as e:
    print(f"O navegador não abriu, mas o teste rodou. Erro visual: {e}")

1. Atualizando a Suite de Testes...
2. Preparando o Checkpoint...
3. Rodando Validação...


Calculating Metrics: 100%|██████████| 39/39 [00:00<00:00, 130.52it/s]


4. Abrindo Relatório...
✅ Sucesso! Verifique a aba do navegador.
